In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('./북한 기후 시간단위/북한 1996(시간).csv', encoding='euckr')
df.head(2)

,지점,일시,전운량(할),최저운고(할),중하층운량(할),하층운형,중층운형,상층운형,기온(°C),풍향(36방위),풍속(m/s),해면기압(hPa),현지기압(hPa),12시간강수량(mm),상대습도(%),이슬점온도(°C),시정(10m)
0,3,1996-01-01 03:00,0,NaN,NaN,NaN,NaN,NaN,-9.8,32.0,5.0,1017.4,NaN,NaN,44.0,-19.9,2000.0
1,3,1996-01-01 06:00,0,NaN,NaN,NaN,NaN,NaN,-9.2,32.0,3.0,1019.0,NaN,NaN,42.0,-19.8,2000.0


In [4]:
df.rename(columns={'지점':'지점번호'}, inplace=True)

In [5]:
df.rename(columns={'풍향(36방위)':'풍향'}, inplace=True)

In [5]:
df.head

<bound method NDFrame.head of        지점번호                일시  전운량(할)  최저운고(할)  중하층운량(할)  하층운형  중층운형  상층운형  \
0         3  1996-01-01 03:00       0      NaN       NaN   NaN   NaN   NaN   
1         3  1996-01-01 06:00       0      NaN       NaN   NaN   NaN   NaN   
2         3  1996-01-01 09:00       0      NaN       NaN   NaN   NaN   NaN   
3         3  1996-01-01 12:00       0      NaN       NaN   NaN   NaN   NaN   
4         3  1996-01-01 15:00       0      NaN       NaN   NaN   NaN   NaN   
...     ...               ...     ...      ...       ...   ...   ...   ...   
71923    75  1996-12-18 06:00       0      9.0       NaN   NaN   NaN   NaN   
71924    75  1996-12-18 09:00       0      9.0       NaN   NaN   NaN   NaN   
71925    75  1996-12-18 15:00       0      9.0       NaN   NaN   NaN   NaN   
71926    75  1996-12-19 06:00       0      9.0       NaN   NaN   NaN   NaN   
71927    75  1996-12-21 06:00      10      5.0      10.0   5.0   9.0   9.0   

       기온(°C)    풍향  풍속(m/s)  해면기

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71928 entries, 0 to 71927
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   지점번호         71928 non-null  int64  
 1   일시           71928 non-null  object 
 2   전운량(할)       71928 non-null  int64  
 3   최저운고(할)      69784 non-null  float64
 4   중하층운량(할)     45032 non-null  float64
 5   하층운형         45138 non-null  float64
 6   중층운형         36168 non-null  float64
 7   상층운형         25371 non-null  float64
 8   기온(°C)       71727 non-null  float64
 9   풍향           71916 non-null  float64
 10  풍속(m/s)      71901 non-null  float64
 11  해면기압(hPa)    63993 non-null  float64
 12  현지기압(hPa)    481 non-null    float64
 13  12시간강수량(mm)  3906 non-null   float64
 14  상대습도(%)      71333 non-null  float64
 15  이슬점온도(°C)    71769 non-null  float64
 16  시정(10m)      71845 non-null  float64
dtypes: float64(14), int64(2), object(1)
memory usage: 9.3+ MB


In [7]:
df['지점번호'].unique()

array([ 3,  5,  8, 14, 16, 20, 22, 25, 28, 31, 35, 37, 39, 41, 46, 50, 52,
       55, 58, 60, 61, 65, 67, 68, 69, 70, 75], dtype=int64)

In [6]:
df['일시'] = df['일시'].apply(lambda x:x.split()[0])

In [7]:
df['풍향'].fillna(99,inplace=True)
# df['풍향'].unique()


In [8]:
df['풍향'] = df['풍향'].apply(lambda x : 99 if x > 36 else x)
# df['풍향'].unique()

In [9]:
import math
df['x_axis'] = df['풍향'].apply(lambda x: math.sin(math.radians(x*10)) if 0<x<37 else x)
df['y_axis'] = df['풍향'].apply(lambda x: math.cos(math.radians(x*10)) if 0<x<37 else x)

In [10]:
def wind_direct(s):
    if s.value_counts().head(1).index == 99:
        return 99
    else:
        return s[s.values != 99].mean()
    

tdf = df.groupby(['지점번호','일시'])['x_axis', 'y_axis'].agg(wind_direct).reset_index()

In [11]:
len(tdf)

9745

In [12]:
tmplst = []
for ind in range(len(tdf)):
    if tdf.loc[ind,'x_axis'] == 99:
        tmplst.append(1000)
    else:
        tmplst.append(math.degrees(math.atan(tdf.loc[ind,'x_axis']/tdf.loc[ind,'y_axis'])))
tdf['일일 풍향']=tmplst
tdf

,지점번호,일시,x_axis,y_axis,일일 풍향
0,3,1996-01-01,-1.836536e-01,0.361727,-2.691751e+01
1,3,1996-01-02,-4.163336e-17,0.125000,-1.908333e-14
2,3,1996-01-03,-6.051917e-01,0.787750,-3.753338e+01
3,3,1996-01-04,-1.606969e-01,0.316511,-2.691751e+01
4,3,1996-01-05,-4.465155e-02,-0.095756,2.500000e+01
...,...,...,...,...,...
9740,75,1996-12-15,0.000000e+00,0.000000,NaN
9741,75,1996-12-17,-5.000000e-01,0.500000,-4.500000e+01
9742,75,1996-12-18,-7.349232e-01,0.085505,-8.336373e+01
9743,75,1996-12-19,0.000000e+00,0.000000,NaN


In [13]:
tdf['일일 풍향'] = tdf['일일 풍향'].apply(lambda x: round(x+360,ndigits=-1) if x<0 else round(x,ndigits=-1)).fillna(0)

In [75]:
tdf[tdf['일일 풍향'].isna()]

,지점번호,일시,x_axis,y_axis,일일 풍향


In [76]:
df_tt = df.groupby(['지점번호', '일시']).agg({'기온(°C)':['mean','max','min'], '상대습도(%)':'mean', 
                                        '풍속(m/s)':'mean', '12시간강수량(mm)':'sum','해면기압(hPa)':'mean',
                                        '이슬점온도(°C)':'mean'})

df_tt.columns = ['평균기온','최고기온','최저기온','습도','풍속','강수량','해면기압','이슬점 온도']
df_city = df_tt.reset_index()

In [77]:
df_city

,지점번호,일시,평균기온,최고기온,최저기온,습도,풍속,강수량,해면기압,이슬점 온도
0,3,1996-01-01,-6.542857,-1.5,-9.8,47.000,2.571429,0.0,1018.785714,-16.4000
1,3,1996-01-02,-4.687500,1.5,-10.4,75.500,0.500000,0.0,1014.662500,-8.6250
2,3,1996-01-03,-7.400000,-1.1,-11.6,57.625,8.250000,0.0,1010.962500,-14.6000
3,3,1996-01-04,-8.862500,-5.0,-12.0,48.375,2.375000,0.0,1020.400000,-18.2375
4,3,1996-01-05,-6.300000,-0.8,-10.5,79.625,0.500000,0.0,1021.712500,-9.4250
...,...,...,...,...,...,...,...,...,...,...
9740,75,1996-12-15,-3.600000,-3.6,-3.6,89.000,0.000000,0.0,1028.050000,-5.1500
9741,75,1996-12-17,-0.050000,3.7,-3.8,73.500,3.500000,0.0,1022.850000,-4.2000
9742,75,1996-12-18,-8.550000,-5.6,-11.3,55.750,2.500000,0.0,1028.575000,-16.6750
9743,75,1996-12-19,-11.500000,-11.5,-11.5,91.000,0.000000,0.0,1027.900000,-12.7000


In [78]:
df_city = pd.concat([df_city,tdf['일일 풍향']],axis=1)

In [79]:
df_city

,지점번호,일시,평균기온,최고기온,최저기온,습도,풍속,강수량,해면기압,이슬점 온도,일일 풍향
0,3,1996-01-01,-6.542857,-1.5,-9.8,47.000,2.571429,0.0,1018.785714,-16.4000,330.0
1,3,1996-01-02,-4.687500,1.5,-10.4,75.500,0.500000,0.0,1014.662500,-8.6250,360.0
2,3,1996-01-03,-7.400000,-1.1,-11.6,57.625,8.250000,0.0,1010.962500,-14.6000,320.0
3,3,1996-01-04,-8.862500,-5.0,-12.0,48.375,2.375000,0.0,1020.400000,-18.2375,330.0
4,3,1996-01-05,-6.300000,-0.8,-10.5,79.625,0.500000,0.0,1021.712500,-9.4250,20.0
...,...,...,...,...,...,...,...,...,...,...,...
9740,75,1996-12-15,-3.600000,-3.6,-3.6,89.000,0.000000,0.0,1028.050000,-5.1500,0.0
9741,75,1996-12-17,-0.050000,3.7,-3.8,73.500,3.500000,0.0,1022.850000,-4.2000,320.0
9742,75,1996-12-18,-8.550000,-5.6,-11.3,55.750,2.500000,0.0,1028.575000,-16.6750,280.0
9743,75,1996-12-19,-11.500000,-11.5,-11.5,91.000,0.000000,0.0,1027.900000,-12.7000,0.0


In [80]:
no = [5, 22, 16, 20, 14, 39, 46, 31, 41, 25, 3, 8]

In [81]:
li_c=[]

for i in df_city['지점번호']:
    if i in no:
        li_c.append('n2')
    else:
        li_c.append('n1')
        
li_c

['n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',
 'n2',

In [82]:
len(li_c)

9745

In [83]:
df_city['군집'] = li_c

In [84]:
df_city

,지점번호,일시,평균기온,최고기온,최저기온,습도,풍속,강수량,해면기압,이슬점 온도,일일 풍향,군집
0,3,1996-01-01,-6.542857,-1.5,-9.8,47.000,2.571429,0.0,1018.785714,-16.4000,330.0,n2
1,3,1996-01-02,-4.687500,1.5,-10.4,75.500,0.500000,0.0,1014.662500,-8.6250,360.0,n2
2,3,1996-01-03,-7.400000,-1.1,-11.6,57.625,8.250000,0.0,1010.962500,-14.6000,320.0,n2
3,3,1996-01-04,-8.862500,-5.0,-12.0,48.375,2.375000,0.0,1020.400000,-18.2375,330.0,n2
4,3,1996-01-05,-6.300000,-0.8,-10.5,79.625,0.500000,0.0,1021.712500,-9.4250,20.0,n2
...,...,...,...,...,...,...,...,...,...,...,...,...
9740,75,1996-12-15,-3.600000,-3.6,-3.6,89.000,0.000000,0.0,1028.050000,-5.1500,0.0,n1
9741,75,1996-12-17,-0.050000,3.7,-3.8,73.500,3.500000,0.0,1022.850000,-4.2000,320.0,n1
9742,75,1996-12-18,-8.550000,-5.6,-11.3,55.750,2.500000,0.0,1028.575000,-16.6750,280.0,n1
9743,75,1996-12-19,-11.500000,-11.5,-11.5,91.000,0.000000,0.0,1027.900000,-12.7000,0.0,n1
